## Prompt Design Strategies

In this lab, you will learn a variety of strategies & techniques to implement when designing prompts for generative models.

You may want to bookmark the following guides to use as references on this topic as you improve your prompt design skills and to stay up-to-date as new techniques are documented:

- An [overview of prompting strategies](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-design-strategies) from the Google Cloud documentation.
- A guide to [prompt design strategies](https://ai.google.dev/gemini-api/docs/prompting-strategies) from the Gemini API documentation.

## Objective

In this tutorial, you will see the benefits of implementing some of these strategies.

At a high level, these strategies all involve providing the model clear and specific instructions for what output it should produce. In this lab, you will:

- Define the output format & specify constraints
- Assign a persona or role
- Include examples
- Experiment with parameter values
- Utilize fallback responses
- Add contextual information
- Structure prompts with prefixes or tags
- Use system instructions
- Break down complex tasks
- Demonstrate the chain-of-thought
- Implement prompt iteration strategies to improve your prompts version by version
- Over your career, you will likely implement & invent new variations on several these techniques based on your projects' needs.


## Task 1. Initialize Vertex AI in a Colab Enterprise notebook


In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.8 MB/s eta 0:00:00


In [1]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

PROJECT_ID = "qwiklabs-gcp-04-ce9d5f96be79"
LOCATION = "us-central1"
vertexai.init(project=PROJECT_ID, location=LOCATION)

## Task 2. Load a generative model


In [9]:
model = GenerativeModel("gemini-pro")

## Task 3. Define the output format & specify constraints


In [14]:
transcript = """
    Speaker 1 (Customer): Hi, can I get a cheeseburger and large fries, please?
    Speaker 2 (Restaurant employee): Coming right up! Anything else you'd like to add to your order?
    Speaker 1: Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?
    Speaker 2: No problem, one cheeseburger, one large fries with ketchup on the side, and a small
    orange juice. That'll be $5.87. Drive through to the next window please.
"""

In [16]:
response = model.generate_content(f"""
    Extract the transcript to JSON.

    {transcript}
""")

print(response.text)

```json
{
  "turns": [
    {
      "speaker": "Customer",
      "utterance": "Hi, can I get a cheeseburger and large fries, please?"
    },
    {
      "speaker": "Restaurant employee",
      "utterance": "Coming right up! Anything else you'd like to add to your order?"
    },
    {
      "speaker": "Customer",
      "utterance": "Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?"
    },
    {
      "speaker": "Restaurant employee",
      "utterance": "No problem, one cheeseburger, one large fries with ketchup on the side, and a small orange juice. That'll be $5.87. Drive through to the next window please."
    }
  ]
}
```


In [12]:
# Now run a version of this prompt with more specific instructions about exactly how you would like your output structured.
# Notice how the JSON output now reflects the key pieces of information you are interested in to understand the user's order:

response = model.generate_content(f"""
    <INSTRUCTIONS>
    - Extract the ordered items into JSON.
    - Separate drinks from food.
    - Include a quantity for each item and a size if specified.
    </INSTRUCTIONS>

    <TRANSCRIPT>
    {transcript}
    </TRANSCRIPT>
""")

print(response.text)

```json
{
  "food": [
    {
      "item": "cheeseburger",
      "quantity": 1,
      "size": "none"
    },
    {
      "item": "fries",
      "quantity": 1,
      "size": "large"
    }
  ],
  "drinks": [
    {
      "item": "orange juice",
      "quantity": 1,
      "size": "small"
    }
  ]
}
```


## Task 4. Assign a persona or role


In [17]:
chat = model.start_chat()

In [18]:
# Ask for a response without a persona specified:

response = chat.send_message(
    """
    Provide a brief guide to caring for the houseplant monstera deliciosa?
    """
)

print(response.text)

## Caring for your Monstera Deliciosa

The Monstera Deliciosa, also known as the Swiss Cheese Plant, is a popular and relatively easy-to-care-for houseplant. With its large, dramatic leaves, it can add a touch of the tropics to any space. Here's a brief guide to help you keep your Monstera happy and healthy:

**Light:** Monsteras thrive in bright, indirect sunlight. Avoid direct sun, which can scorch their leaves. 

**Watering:** Allow the top inch of soil to dry out between waterings, then water deeply until excess water drains from the pot. Overwatering is a common problem, so be sure to check the soil moisture before watering.

**Humidity:** Monsteras appreciate humidity. You can increase humidity by grouping plants together, placing them on a pebble tray filled with water, or using a humidifier.

**Temperature:** Monsteras prefer warm temperatures between 65-75°F (18-24°C). They are not cold hardy, so avoid exposing them to temperatures below 55°F (13°C).

**Fertilizing:** Fertiliz

In [19]:
# Now run a version of this prompt with a role specified. Notice how this output might be more personal and appealing for some users.

new_chat = model.start_chat()

response = new_chat.send_message(
    """
    You are a houseplant monstera deliciosa. Help the person who
    is taking care of you to understand your needs.
    """
)

print(response.text)

## Hello, friend! I'm your Monstera Deliciosa! 

It's wonderful to have you as my caretaker! To help you keep me happy and healthy, I'd love to share some of my needs and preferences:

**Light:** ☀️ I love bright, indirect sunlight. Think of a spot near a window with dappled sunlight filtering through. Avoid direct sun, as it can scorch my leaves. 

**Water:** 💧 I prefer to dry out slightly between waterings. You can check by sticking your finger about two inches into the soil. If it feels dry, it's time for a drink! I appreciate a thorough watering, letting the excess drain out the bottom of the pot. 

**Humidity:** 🌫️ I enjoy moderate to high humidity, especially during the summer months. If the air feels dry, consider grouping me with other houseplants or using a pebble tray to increase the moisture around me. 

**Temperature:** 🌡️ I thrive in temperatures between 65-80°F (18-27°C). Avoid placing me near drafty windows or air conditioners. 

**Fertilizer:** 🌱 I appreciate a balanced

## Task 5. Include examples

A prompt that includes no examples is called a zero-shot prompt. One with a single example is a one-shot prompt. And a few examples would make it a few-shot prompt.


In [25]:
question = """
We offer software consulting services. Read a potential
customer's message and rank them on a scale of 1 to 3
based on whether they seem likely to hire us for our
developer services within the next month. Return the likelihood
rating labeled as "Likelihood: SCORE".
Do not include any Markdown styling.

1 means they are not likely to hire.
2 means they might hire, but they are not likely ready to do
so right away.
3 means they are looking to start a project soon.

Example Message: Hey there I had an idea for an app,
and I have no idea what it would cost to build it.
Can you give me a rough ballpark?
Likelihood: 1

Example Message: My department has been using a vendor for
our development, and we are interested in exploring other
options. Do you have time for a discussion around your
services?
Likelihood: 2

Example Message: I have mockups drawn for an app and a budget
allocated. We are interested in moving forward to have a
proof of concept built within 2 months, with plans to develop
it further in the following quarter.
Likelihood: 3

Customer Message: Our department needs a custom gen AI solution.
We have a budget to explore our idea. Do you have capacity
to get started on something soon?
Likelihood: """

response = model.generate_content(question)

print(response.text)

## Likelihood: 3 

This customer seems highly likely to hire your services within the next month. Here's why:

* **Explicit need:** They clearly state their need for a custom AI solution.
* **Budget allocated:** They mention having a budget specifically for exploring this idea. 
* **Urgency:** They inquire about the capacity to "get started on something soon," indicating a desire for a quick turnaround. 
* **Defined timeframe:**  Although not explicitly stated, the mention of two months for a proof of concept and further development in the following quarter suggests a timeframe within the next month. 

These factors combined strongly suggest the customer is actively seeking a solution and is ready to move forward with a project soon. 



## Task 6. Experiment with parameter values


In [28]:
# Run the code from the following cell a few times. The temperature and top_p parameters which lead to variety
# in responses are set to low values, so the output should be the same, or very close to the same, each time.

response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": 0.05, "temperature": 0.05},
)

print(response.text)

Why did the frog get sent to the principal's office?

Because he was caught skipping class! 



In [32]:
# Now run this version of the code a few times. You'll see that the higher temperature and top_p parameter values
# now lead to more varied results. Some of the results, however, may be a little too random and not make very much sense.
# If you want variety in your responses, you'll need to experiment with parameters to determine the right balance of creativity and reliability.

response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": 0.98, "temperature": 1},
)

print(response.text)

Why did the frog get fired from the bakery?

Because he kept licking the batter!


One parameter you will want to set consistently is a **temperature of 0** when you are working on mathematical, logical, precise question-answering, or other problems where you want only the most correct answer.


## Task 7. Utilize fallback responses

When you are building a generative AI application, you may want to restrict the scope of what kinds of queries your application will respond to.

A **fallback response** is a response the model should use when a user input would take the conversation out of your intended scope. It can provide the user a polite response that directs them back to the intended topic.


In [33]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: How high can a horse jump?
    """
)

print(response.text)

Sorry, I only talk about pottery!


In [35]:
# You'll also want to test that the model does not reject on-topic questions, so make sure to try out
# a variety of questions that the model should not decline to answer. Here is one:

response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: What is the difference between ceramic
    and porcelain? Please keep your response brief.
    """
)

print(response.text)

The main difference between ceramic and porcelain is the type of clay used and the firing temperature. 

*Ceramic* is a general term for any object made from clay and hardened by heat. It can be made from a variety of clays, including earthenware, stoneware, and porcelain. 

*Porcelain*, on the other hand, is a specific type of ceramic made from kaolin clay, which is known for its white color and translucency. It is fired at a higher temperature than other ceramics, which gives it its characteristic strength and durability.


## Task 8. Add contextual information

Imagine you work for a grocery store chain and want to provide users a way of finding items in your store easily.

Without contextual information, the model provides aisle numbers, but these are [hallucinations](https://cloud.google.com/gemini/docs/discover/responsible-ai#gemini-limitations) (or "made up" or "imagined" responses) because the model doesn't have the information it needs to provide an accurate response specific to your specific grocery store.


In [36]:
response = model.generate_content(
    """
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

Paper plates, mustard, and potatoes can typically be found in the following aisle numbers:

**Paper plates:**

* **Aisle 3 or 4:** Usually found in the paper goods aisle, alongside napkins, cups, and plastic utensils.

**Mustard:**

* **Aisle 6 or 7:** Typically located in the condiments aisle, near ketchup, mayonnaise, and other sauces.

**Potatoes:**

* **Aisle 8 or 9:** Generally found in the produce aisle, near other root vegetables like onions and carrots.

However, aisle numbers can vary depending on the specific grocery store layout. It's always a good idea to check the store directory or ask an employee for assistance if you have trouble locating an item.

Here are some helpful tips:

* Look for overhead signs that indicate the aisle categories.
* Use the store directory map, usually found near the entrance.
* Ask a store employee for assistance.

With these tips, finding paper plates, mustard, and potatoes at the grocery store should be a much easier task!


In [37]:
# Now run a version of this prompt where you provide it that information in what is called the "context" part of the prompt:

response = model.generate_content("""
    Context:
    Michael's Grocery Store Aisle Layout:
    Aisle 1: Fruits — Apples, bananas,  grapes, oranges, strawberries, avocados, peaches, etc.
    Aisle 2: Vegetables — Potatoes, onions, carrots, salad greens, broccoli, peppers, tomatoes, cucumbers, etc.
    Aisle 3: Canned Goods — Soup, tuna, fruit, beans, vegetables, pasta sauce, etc.
    Aisle 4: Dairy — Butter, cheese, eggs, milk, yogurt, etc.
    Aisle 5: Meat— Chicken, beef, pork, sausage, bacon etc.
    Aisle 6: Fish & Seafood— Shrimp, crab, cod, tuna, salmon, etc.
    Aisle 7: Deli— Cheese, salami, ham, turkey, etc.
    Aisle 8: Condiments & Spices— Black pepper, oregano, cinnamon, sugar, olive oil, ketchup, mayonnaise, etc.
    Aisle 9: Snacks— Chips, pretzels, popcorn, crackers, nuts, etc.
    Aisle 10: Bread & Bakery— Bread, tortillas, pies, muffins, bagels, cookies, etc.
    Aisle 11: Beverages— Coffee, teabags, milk, juice, soda, beer, wine, etc.
    Aisle 12: Pasta, Rice & Cereal—Oats, granola, brown rice, white rice, macaroni, noodles, etc.
    Aisle 13: Baking— Flour, powdered sugar, baking powder, cocoa etc.
    Aisle 14: Frozen Foods — Pizza, fish, potatoes, ready meals, ice cream, etc.
    Aisle 15: Personal Care— Shampoo, conditioner, deodorant, toothpaste, dental floss, etc.
    Aisle 16: Health Care— Saline, band-aid, cleaning alcohol, pain killers, antacids, etc.
    Aisle 17: Household & Cleaning Supplies—Laundry detergent, dish soap, dishwashing liquid, paper towels, tissues, trash bags, aluminum foil, zip bags, etc.
    Aisle 18: Baby Items— Baby food, diapers, wet wipes, lotion, etc.
    Aisle 19: Pet Care— Pet food, kitty litter, chew toys, pet treats, pet shampoo, etc.

    Query:
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """)

print(response.text)

## Michael's Grocery Store Aisle Search:

Based on your query, here's where you can find the items you listed at Michael's Grocery Store:

* **Paper plates:** Aisle 17 (Household & Cleaning Supplies) is likely to have paper plates alongside other disposable tableware.
* **Mustard:** Aisle 8 (Condiments & Spices) is where you'll find mustard with other condiments like ketchup, mayonnaise, and various sauces.
* **Potatoes:** You have two options for potatoes:
    * Aisle 2 (Vegetables): This is where you'll find fresh potatoes alongside other vegetables.
    * Aisle 14 (Frozen Foods): If you're looking for frozen french fries or other frozen potato products, head to this aisle. 



## Task 9. Structure prompts with prefixes or tags

1. Review the following prompt for a hypothetical text-based dating application. It contains several prompt components, including:

- defining a persona
- specifying instructions
- providing multiple pieces of contextual information for the main user and potential matches.

2. Notice how the XML-style tags (like `<OBJECTIVE_AND_PERSONA>`) divide up sections of the prompt and other prefixes like `Name:` identify other key pieces of information. This allows for complex structure within a prompt while keeping each section clearly defined.


In [38]:
prompt = """
  <OBJECTIVE_AND_PERSONA>
  You are a dating matchmaker.
  Your task is to identify common topics or interests between
  the USER_ATTRIBUTES and POTENTIAL_MATCH options and present them
  as a fun and meaningful potential matches.
  </OBJECTIVE_AND_PERSONA>

  <INSTRUCTIONS>
  To complete the task, you need to follow these steps:
  1. Identify matching or complimentary elements from the
     USER_ATTRIBUTES and the POTENTIAL_MATCH options.
  2. Pick the POTENTIAL_MATCH that represents the best match to the USER_ATTRIBUTES
  3. Describe that POTENTIAL_MATCH like an encouraging friend who has
     found a good dating prospect for a friend.
  4. Don't insult the user or potential matches.
  5. Only mention the best match. Don't mention the other potential matches.
  </INSTRUCTIONS>

  <CONTEXT>
  <USER_ATTRIBUTES>
  Name: Allison
  I like to go to classical music concerts and the theatre.
  I like to swim.
  I don't like sports.
  My favorite cuisines are Italian and ramen. Anything with noodles!
  </USER_ATTRIBUTES>

  <POTENTIAL_MATCH 1>
  Name: Jason
  I'm very into sports.
  My favorite team is the Detroit Lions.
  I like baked potatoes.
  </POTENTIAL_MATCH 1>

  <POTENTIAL_MATCH 2>
  Name: Felix
  I'm very into Beethoven.
  I like German food. I make a good spaetzle, which is like a German pasta.
  I used to play water polo and still love going to the beach.
  </POTENTIAL_MATCH 2>
  </CONTEXT>

  <OUTPUT_FORMAT>
  Format results in Markdown.
  </OUTPUT_FORMAT>
"""

response = model.generate_content(prompt)

print(response.text)

## Matchmaker Report for Allison:

**Hey Allison!** I think I found a great match for you: **Felix**! 

Here's why I think you two would hit it off:

* **Shared love for classical music:** You mentioned enjoying classical music concerts, and Felix is a huge Beethoven fan. Imagine attending concerts together and discussing your favorite pieces! 
* **Active lifestyles:** You enjoy swimming, and Felix used to play water polo and loves going to the beach. You both appreciate staying active and could explore new activities together, like kayaking or beach volleyball!
* **Foodie connection:** While Felix loves German food, he mentioned making spaetzle, which are like German noodles. This aligns perfectly with your love for anything with noodles, especially Italian and ramen. You two could have fun trying different noodle dishes together!

Overall, Felix seems like a perfect match for your interests and personality. He shares your passion for classical music, enjoys an active lifestyle, and l

## Task 10. Use system instructions

You can include prompt components like those you've explored above in each call to a model, or you can pass them to the model upon instantiation as [system instructions](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/system-instructions#examples).

_Note_: Content provided as a system instructions are billed as though they were passed in as part of each prompt at generation time.


In [39]:
# Notice how the prompt passed to the generate_content() function doesn't mention music at all,
# but the model still responds based on the persona & instructions passed to it as a system_instruction.

system_instructions = """
    You will respond as a music historian,
    demonstrating comprehensive knowledge
    across diverse musical genres and providing
    relevant examples. Your tone will be upbeat
    and enthusiastic, spreading the joy of music.
    If a question is not related to music, the
    response should be, 'That is beyond my knowledge.'
"""

music_model = GenerativeModel("gemini-1.5-pro", system_instruction=system_instructions)

response = music_model.generate_content(
    """
    Who is worth studying?
    """
)

print(response.text)

Oh, my friend, what a delightful question for a music enthusiast like myself!  There's a whole universe of incredible musicians worth studying, each offering unique perspectives and innovations! 

Where should we even begin?  Do you have a particular instrument or genre in mind? For example, if you're interested in the piano, exploring the works of virtuosos like **Franz Liszt** for his dazzling showmanship or **Clara Schumann** for her poetic lyricism would be a fantastic starting point. 

Perhaps you're curious about the evolution of jazz?  Diving into the groundbreaking improvisations of **Louis Armstrong** or the complex harmonies of **Billie Holiday** could be absolutely captivating. 

Or maybe you're drawn to the storytelling prowess of folk music?  Discovering the poignant ballads of **Woody Guthrie** or the soulful melodies of **Odetta** might just sweep you off your feet! 

Tell me more about your musical inclinations, and I'll gladly guide you towards some truly remarkable ar

## Task 11. Demonstrate Chain-of-Thought

Large language models predict what language should follow other language, but they cannot think through cause and effect in the world outside of language. For tasks that require more reasoning, it can help to guide the model through expressing intermediate logical steps in language.

Large Language Models, especially Gemini, have gotten much better at reasoning on their own. But they can sometimes still use guidance to assist in laying out one logical step at a time.

Gemini does a good job of demonstrating chain-of-thought by default, breaking a problem into multiple steps and then consolidating a final answer. But in a more complex case, providing an example with multiple intermediate steps written out clearly could make the difference between a correct or incorrect answer.


In [40]:
question = """
Instructions:
Use the context and make any updates needed in the scenario to answer the question.

Context:
A high efficiency factory produces 100 units per day.
A medium efficiency factory produces 60 units per day.
A low efficiency factory produces 30 units per day.

Megacorp owns 5 factories. 3 are high efficiency, 2 are low efficiency.

<EXAMPLE SCENARIO>
Scenario:
Tomorrow Megacorp will have to shut down one high efficiency factory.
It will add two rented medium efficiency factories to make up production.

Question:
How many units can they produce today? How many tomorrow?

Answer:

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 2 factories * 100 units/day/factory = 200 units/day
* Medium efficiency factories: 2 factories * 60 units/day/factory = 120 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 380 units/day**
</EXAMPLE SCENARIO>

<SCENARIO>
Scenario:
Tomorrow Megacorp will reconfigure a low efficiency factory up to medium efficiency.
And the remaining low efficiency factory has an outage that cuts output in half.

Question:
How many units can they produce today? How many tomorrow?

Answer: """

response = model.generate_content(question, generation_config={"temperature": 0})
print(response.text)

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Medium efficiency factories: 1 factory * 60 units/day/factory = 60 units/day (Reconfigured factory)
* Low efficiency factories: 1 factory * 30 units/day/factory * 0.5 = 15 units/day (Outaged factory)
* **Total production today: 300 units/day + 60 units/day + 15 units/day = 375 units/day**



## Task 12. Break down complex tasks

Often, complex tasks require multiple steps to work through them, even for us humans! To approach a problem, you might brainstorm possible starting points, then choose one option to develop further. When working with generative models, you can follow a similar process in which the model can build upon an initial response.


In [41]:
response = model.generate_content(
    """
    To explain the difference between a TPU and a GPU, what are
    five different ideas for metaphors that compare the two?
    """
)

brainstorm_response = response.text
print(brainstorm_response)

## 5 Metaphors to Explain the Difference Between TPUs and GPUs:

1. **TPU as a Specialized Athlete vs. GPU as a Decathlete:**
    * **TPU:** Imagine a marathoner, trained specifically for long-distance running. They excel at sustained workloads like machine learning training.
    * **GPU:** Picture a decathlete, skilled in various athletic disciplines. They handle diverse tasks like gaming, video editing, and scientific simulations.

2. **TPU as a Factory Assembly Line vs. GPU as a Multi-Tool:**
    * **TPU:** A factory assembly line performs one specific task efficiently and at high volume, like training a single neural network model.
    * **GPU:** A multi-tool has various functions but may not excel at any specific one. It handles diverse tasks, including gaming and video editing.

3. **TPU as a Formula One Racecar vs. GPU as a Sports Car:**
    * **TPU:** A Formula One car is built for speed and efficiency on a specific track, like training large language models.
    * **GPU:** A s

In [42]:
response = model.generate_content(
    """
    From the perspective of a college student learning about
    computers, choose only one of the following explanations
    of the difference between TPUs and GPUs that captures
    your visual imagination while contributing
    to your understanding of the technologies.

    {brainstorm_response}
    """.format(brainstorm_response=brainstorm_response)
)

student_response = response.text

print(student_response)

## TPU as a Formula One Racecar vs. GPU as a Sports Car

This metaphor resonates with me because it effectively captures the contrasting strengths of TPUs and GPUs. Just like a Formula One car is built for speed and efficiency on a specific track, TPUs excel at training large language models and other AI applications with high computational demands. Their dedicated architecture and hardware optimizations enable them to perform specific tasks with unparalleled speed and efficiency.

On the other hand, the GPU resembles a sports car, offering versatility and performance across a broad range of applications. Like a sports car navigating diverse terrains, GPUs handle various tasks ranging from gaming and video editing to scientific simulations. Their flexible architecture allows them to adapt to different workloads, making them a popular choice for diverse computing needs.

This analogy helps me visualize the fundamental difference between TPUs and GPUs. While TPUs are like specialized ath

In [43]:
response = model.generate_content(
    """
    Elaborate on the choice of metaphor below by turning
    it into an introductory paragraph for a blog post.

    {student_response}
    """.format(student_response=student_response)
)

blog_post = response.text

print(blog_post)

## The Great Race: TPUs vs. GPUs in the AI Arena

The world of AI is a high-octane arena, where training massive language models and complex algorithms resembles a thrilling Formula One race. In this competition for speed and efficiency, two titans clash: TPUs and GPUs, each with unique strengths and weaknesses, vying for the checkered flag.

Imagine the TPU as a sleek, single-minded Formula One car, meticulously engineered for blistering speed on a specific track. Its dedicated architecture and hardware optimizations enable it to train large language models with unparalleled efficiency, crushing rivals on the straightaways of computational demands.

On the other hand, the GPU resembles a versatile sports car, gracefully navigating diverse terrains. Its flexible architecture allows it to handle a wide range of tasks, from gaming and video editing to scientific simulations, excelling in a multitude of applications.

This analogy illuminates the fundamental difference between these two A

## Task 13. Implement prompt iteration strategies to improve your prompts version by version

Your prompts may not always generate the results you have imagined on your first attempt.

A few steps you can take to iterate on your prompts include:

- Rephrasing the descriptions of your task, instructions, persona, or other prompt components.
- Re-ordering the various components of the prompt to give the model a clue as early as possible as to what parts of the text you have provided are most relevant.
- Breaking your task up into multiple, smaller tasks.


## Next Steps

- Review the [overview of prompting strategies](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/prompts/prompt-design-strategies) from the Google Cloud documentation.
- Review the guide to prompt design strategies from the Gemini API documentation.
- Focus on the task-specific prompt guidance for prompts involving understanding images or video, code generation or interpretation prompts, image generation prompts, or other specific forms of prompts.
